## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [18]:
# 데이터 로드
DI_train = pd.read_csv('../data/DI_train_dataset_39.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_39.csv')

In [19]:
# ID 열을 제외한 특성과 타겟 변수 분리
DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [20]:
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

DI_X shape: (6289, 22)
DI_test shape: (2176, 22)


### 인코딩 

In [21]:
DI_categorical_columns = [
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [22]:
# 모든 범주형 변수를 문자열로 변환
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [23]:
# 데이터 분할
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [24]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.00005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 200.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 200.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-18 19:18:03,888] A new study created in memory with name: no-name-2f73dba7-abcb-4837-bce9-82b7f4376f46


[I 2025-02-18 19:18:04,732] Trial 0 finished with value: 0.7081137225951553 and parameters: {'n_estimators': 2062, 'num_leaves': 4477, 'learning_rate': 0.22400159066152517, 'min_child_samples': 174, 'reg_alpha': 31.81584095627108, 'reg_lambda': 104.22997907341424}. Best is trial 0 with value: 0.7081137225951553.
[I 2025-02-18 19:18:05,247] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 1240, 'num_leaves': 3501, 'learning_rate': 0.3383741233195419, 'min_child_samples': 93, 'reg_alpha': 192.36623939202119, 'reg_lambda': 62.403701079844275}. Best is trial 0 with value: 0.7081137225951553.
[I 2025-02-18 19:18:05,895] Trial 2 finished with value: 0.6457356888194528 and parameters: {'n_estimators': 1997, 'num_leaves': 2742, 'learning_rate': 0.4420742958417083, 'min_child_samples': 88, 'reg_alpha': 0.696768262037839, 'reg_lambda': 0.9206801719344959}. Best is trial 0 with value: 0.7081137225951553.
[I 2025-02-18 19:18:06,749] Trial 3 finished with value: 0.7063854504666907 

Best trial:
  Value: 0.7169010134999799
  Params: 
    n_estimators: 703
    num_leaves: 4344
    learning_rate: 0.11319732648723346
    min_child_samples: 108
    reg_alpha: 4.902965791042313
    reg_lambda: 138.2191782322712


In [9]:
# import optuna
# import lightgbm as lgb
# from sklearn.metrics import roc_auc_score

# # 목적 함수 정의
# def objective(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 500, 9000),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
#         'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
#         'subsample': trial.suggest_float('subsample', 0.2, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),

#         'random_state': 42,
#         'scale_pos_weight': 6.757,
#         'objective': 'binary',
#         'metric': 'auc',
#         'verbose': -1     
#     }

#     model = lgb.LGBMClassifier(**param)
#     model.fit(DI_X_train, DI_y_train)
    
#     y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
#     auc = roc_auc_score(DI_y_test, y_pred_proba)
#     return auc

# # Optuna 스터디 생성 및 최적화 실행
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=2000)

# # 최적의 하이퍼파라미터 출력
# print("Best trial:")
# trial = study.best_trial
# print(f"  Value: {trial.value}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

.